## Twitter API with Tweepy

In [2]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
CONSUMER_API_KEY = 'your consumer api key'
CONSUMER_API_SECRET = 'your consumer api secret'
ACCESS_TOKEN ='your token'
ACCESS_TOKEN_SECRET = 'your token secret'
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [4]:
# Function to get trending topic by country name
def get_trending_topic_by_country(country, num_trends):
    trends_country = api.trends_available()
    for trend in trends_country:
        if trend['name'] == country:
            country_info = (trend['name'], trend['woeid'])
            print('Country: {}, WOEID: {}'.format(trend['name'], trend['woeid']))
            
    trending_topics = []
    trends_result = api.trends_place(country_info[1])
    for tren in trends_result[0]['trends'][:num_trends]: 
        print(tren['name'])
        trending_topics.append(tren['name'])
        
    return trending_topics

def get_tweets_by_keyword(keywords, num_of_tweets, language, filename='', export_to_csv=False):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
    
    
    if export_to_csv == True:
        exsport_to_csv(df_tweets, filename)
        
    return df_tweets

def get_tweets_by_keyword(keywords, num_of_tweets, language):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
        
    return df_tweets

def remove_unwanted_string(dataframe):
    new_record = []
    tweet_text = dataframe.apply(lambda x: str(x).split())
    for record in tweet_text:
        for text in record:
            if 'RT' in text or '@' in text or '#' in text:
                record.remove(text)
        
        new_record.append(' '.join(record))
    dataframe = new_record
    return dataframe
            
# Function to print sentiments of the sentence
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object
    sid_obj = SentimentIntensityAnalyzer()
    
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary
    # which contains pos, neg, neu, and compoind scores
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict
    
# Function export to CSV
def exsport_to_csv(dataframe, filename):
    print('\nExported to CSV')
    return dataframe.to_csv(filename)

In [5]:
df_tweets = get_tweets_by_keyword(['corona', 'COVID-19'], 10, 'en')

df_tweets['tweet_text'] = remove_unwanted_string(df_tweets['tweet_text'])

pos, neg, neu, comp = [], [], [], []
for text in df_tweets['tweet_text']:
    sentiment = sentiment_scores(text)
    pos.append(sentiment['pos'])
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    comp.append(sentiment['compound'])
    
df_tweets_sentiment = pd.DataFrame()
# df_tweets_sentiment.columns = ['tweet_text', 'positive', 'negative', 'neutral', 'compound']
df_tweets_sentiment['tweet_text'] = df_tweets['tweet_text']
df_tweets_sentiment['positive'] = pos
df_tweets_sentiment['negative'] = neg
df_tweets_sentiment['neutral'] = neu
df_tweets_sentiment['compound'] = comp

In [6]:
df_tweets_sentiment.head()

,tweet_text,positive,negative,neutral,compound
0,@YifatRoth: Our engagements and preferences wi...,0.071,0.000,0.929,0.1531
1,@YifatRoth: Human society has exhausted itself...,0.107,0.099,0.794,0.0516
2,@Just_Screams: Hey I live in a student housing...,0.000,0.000,1.000,0.0000
3,@YifatRoth: Many of our current businesses and...,0.000,0.128,0.872,-0.2960
4,@Indiaaakash: This is how Corona Terr0rist sho...,0.000,0.000,1.000,0.0000


In [7]:
df_tweets_sentiment['tweet_text']

0    @YifatRoth: Our engagements and preferences wi...
1    @YifatRoth: Human society has exhausted itself...
2    @Just_Screams: Hey I live in a student housing...
3    @YifatRoth: Many of our current businesses and...
4    @Indiaaakash: This is how Corona Terr0rist sho...
5    #COVID__19 SHOULD BE BANNED ACROSS ALL MAJOR C...
6    @Richard_Spoor: In more news: Cabinet Minister...
7    Positive Thinking #coronaupdatesindia #lockdow...
8    @ishavidhya: In all 17 Panchayats that our vol...
9    @ishavidhya: In all 17 Panchayats that our vol...
Name: tweet_text, dtype: object